<a href="https://colab.research.google.com/github/s-choung/Matlantis_benchmarking/blob/main/data_save_from_catalysis_hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Following code collect data from catalysis-hub and save in form of json


credit: seokhyun

In [ ]:
!pip install ase

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.7 MB/s eta 0:00:00


In [ ]:
import requests
import pprint
import sys
import string
import json
import io
import copy

import ase.io
import ase.calculators.singlepoint

GRAPHQL = 'http://api.catalysis-hub.org/graphql'

def fetch(query):
    return requests.get(
        GRAPHQL, {'query': query}
    ).json()['data']

In [ ]:
def reactions_from_dataset(pub_id, page_size=40):
    reactions = []
    has_next_page = True
    start_cursor = ''
    page = 0
    while has_next_page:
        data = fetch("""{{
      reactions(pubId: "{pub_id}", first: {page_size}, after: "{start_cursor}") {{
        totalCount
        pageInfo {{
          hasNextPage
          hasPreviousPage
          startCursor
          endCursor 
        }}  
        edges {{
          node {{
            Equation
            reactants
            products
            reactionEnergy
            reactionSystems {{
              name
              systems {{
                energy
                InputFile(format: "json")
              }}
            }}  
          }}  
        }}  
      }}    
    }}""".format(start_cursor=start_cursor,
                 page_size=page_size,
                 pub_id=pub_id,
                ))
        has_next_page = data['reactions']['pageInfo']['hasNextPage']
        start_cursor = data['reactions']['pageInfo']['endCursor']
        page += 1
        print(has_next_page, start_cursor, page_size * page, data['reactions']['totalCount'])
        reactions.extend(map(lambda x: x['node'], data['reactions']['edges']))

    return reactions
dataset_name_is="Gauthierrole2021"
#raw_reactions3 = reactions_from_dataset(dataset_name_is) # oxide
#raw_reactions4 = reactions_from_dataset("MeffordInterpreting2019") # 2d NiFeOOH  
#raw_reactions5 = reactions_from_dataset("BackPrediction2018") # metal + NiFeOOH
#raw_reactions6 = reactions_from_dataset("JuUnraveling2019") # sac 
raw_reactions7 = reactions_from_dataset(dataset_name_is) # sac 
#raw_reactions8 = reactions_from_dataset("RaoResolving2022") # Nickel Antimonate 
import json
import os

def data2json(raw_reactions, dataset_name):
    raw_reactions_json = {"raw_reactions": raw_reactions}
    filename = f"{dataset_name}.json"
    path = os.path.join(os.getcwd(), filename)
    with open(path, 'w') as f:
        json.dump(raw_reactions_json, f)
data2json(raw_reactions7,dataset_name_is)
path_json = os.path.join(os.getcwd(), dataset_name_is+".json")
with open(path_json, 'r') as f:
  data = json.load(f)
pierre=data['raw_reactions']


True YXJyYXljb25uZWN0aW9uOjM5 40 351
True YXJyYXljb25uZWN0aW9uOjc5 80 351
True YXJyYXljb25uZWN0aW9uOjExOQ== 120 351
True YXJyYXljb25uZWN0aW9uOjE1OQ== 160 351
True YXJyYXljb25uZWN0aW9uOjE5OQ== 200 351
True YXJyYXljb25uZWN0aW9uOjIzOQ== 240 351
True YXJyYXljb25uZWN0aW9uOjI3OQ== 280 351
True YXJyYXljb25uZWN0aW9uOjMxOQ== 320 351
False YXJyYXljb25uZWN0aW9uOjM1MA== 360 351


In [ ]:
def aseify_reactions(reactions):
    for i, reaction in enumerate(reactions):
        for j, _ in enumerate(reactions[i]['reactionSystems']):
            with io.StringIO() as tmp_file:
                system = reactions[i]['reactionSystems'][j].pop('systems')
                tmp_file.write(system.pop('InputFile'))
                tmp_file.seek(0)
                atoms = ase.io.read(tmp_file, format='json')
            calculator = ase.calculators.singlepoint.SinglePointCalculator(
                atoms,
                energy=system.pop('energy')
            )
            atoms.set_calculator(calculator)
            #print(atoms.get_potential_energy())
            reactions[i]['reactionSystems'][j]['atoms'] = atoms
        # flatten list further into {name: atoms, ...} dictionary
        reactions[i]['reactionSystems'] = {x['name']: x['atoms']
                                          for x in reactions[i]['reactionSystems']}

In [ ]:
import os

benchmark="Gauthierrole2021"
path_json = os.path.join(os.getcwd(),benchmark+".json")
with open(path_json, 'r') as f:
  data = json.load(f)
dat=data['raw_reactions']
dat = copy.deepcopy(dat)
aseify_reactions(dat)

real_E=[round(dat[i]['reactionEnergy'],3) for i in range(0,len(dat))]

print(real_E)
print(len(real_E))

[0.658, 0.506, 0.549, -0.348, 0.399, 1.18, 0.399, 0.979, -0.335, 1.597, -0.895, 0.382, -0.477, 0.208, -0.23, 0.95, 1.677, 0.106, -0.88, -0.563, -1.797, -1.726, -0.023, 0.99, -0.323, -1.367, -1.995, -1.319, -0.783, -2.159, -1.843, -0.436, 0.401, -1.228, -0.829, -0.485, -1.853, -0.456, -2.241, -0.648, -0.293, -1.76, 0.07, -1.723, -2.763, -2.157, -0.688, -1.815, -1.016, -1.852, -0.836, -0.034, -1.817, -0.434, 1.443, -1.88, -0.585, -0.216, 0.828, -1.622, 0.782, -0.592, 1.568, -1.832, -0.023, -4.738, -3.534, -3.813, -2.257, -3.71, -3.92, -3.495, -0.511, -0.011, -1.065, -0.082, -0.222, -1.1, -1.402, -2.101, -1.03, 0.836, 1.268, -0.04, 0.087, -0.878, -3.09, -0.198, -0.867, -1.145, 0.423, -1.36, -1.935, -2.606, -3.013, -2.08, -2.226, -2.188, -2.328, 0.669, 1.544, 1.453, 0.805, 0.635, -0.722, -1.004, 0.813, 0.132, 1.749, 1.46, 1.788, 0.468, -0.255, 2.74, 1.337, -1.902, -1.713, 0.889, -2.808, -3.596, -1.326, -0.287, -0.392, 1.957, 2.798, -3.119, 0.318, 0.008, 0.983, 1.891, -1.529, 0.522, 1.606, 

In [ ]:
import json
def stoich(data,i):
    data_string =data[i]['reactants']
    prod_string =data[i]['products']
    reactants_list = list(json.loads(data_string).keys())
    values_list = list(json.loads(data_string).values())
    product = list(json.loads(prod_string).keys())
    return values_list, reactants_list, product

def OH_detactor(data):
  oh_list=[]
  for i in range(0,len(data)):
    prod_string =data[i]['products']
    product = list(json.loads(prod_string).keys())
    print(product)
    if len(product)>1:
      print('here')
      if product[1]=='OHstar':
       oh_list.append(i)
  return oh_list

print(OH_detactor(dat))


['H2Ogas', 'CH2star']
here
['COOHstar']
['CHstar', 'H2Ogas']
here
['COstar']
['HCOOstar']
['H2gas', 'OHstar']
here
['Hstar']
['COHstar']
['H2Ogas', 'OCCstar']
here
['Cstar', 'H2Ogas']
here
['H2Ogas', 'CH2star']
here
['COOHstar']
['COstar']
['HCOOstar']
['CHOstar']
['H2gas', 'OHstar']
here
['Cstar', 'H2Ogas']
here
['Hstar']
['H2Ogas', 'CH2star']
here
['COOHstar']
['CHstar', 'H2Ogas']
here
['COstar']
['HCOOstar']
['H2gas', 'OHstar']
here
['Hstar']
['COHstar']
['H2Ogas', 'OCCstar']
here
['Cstar', 'H2Ogas']
here
['COOHstar']
['H2Ogas', 'CH2star']
here
['COstar']
['HCOOstar']
['H2gas', 'OHstar']
here
['CHOstar']
['Cstar', 'H2Ogas']
here
['Hstar']
['H2Ogas', 'CH2star']
here
['COOHstar']
['CHstar', 'H2Ogas']
here
['HCOOstar']
['H2gas', 'OHstar']
here
['COstar']
['Hstar']
['COHstar']
['H2Ogas', 'OCCstar']
here
['Cstar', 'H2Ogas']
here
['COOHstar']
['H2Ogas', 'CH2star']
here
['HCOOstar']
['COstar']
['CHOstar']
['H2gas', 'OHstar']
here
['Cstar', 'H2Ogas']
here
['Hstar']
['H2gas', 'OHstar']
here
